In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from random import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from keras.callbacks import CSVLogger
from keras.layers import BatchNormalization
from sklearn.metrics import f1_score
from gensim.models import KeyedVectors
import gensim
import json
import warnings
import logging, sys
import os
from datetime import datetime
logging.disable(sys.maxsize)
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/task_data/
!ls

In [ ]:
emd_file = '../amazon/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(emd_file, binary=True)
w2v_set = set(word2vec.vocab)

In [79]:
df = pd.read_csv('train_data.csv')
df.drop('path',axis='columns', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True)
df.head()

,transcription,action,object,location
0,Turn on the kitchen lights,activate,lights,kitchen
1,Turn up the temperature,increase,heat,none
2,OK now switch the main language to Chinese,change language,Chinese,none
3,Turn down the bathroom temperature,decrease,heat,washroom
4,Change the language,change language,none,none


In [80]:
act_encoder = OneHotEncoder(sparse=False, dtype='int32', handle_unknown='ignore')
obj_encoder = OneHotEncoder(sparse=False, dtype='int32', handle_unknown='ignore')
loc_encoder = OneHotEncoder(sparse=False, dtype='int32', handle_unknown='ignore')

act_oh = act_encoder.fit_transform(np.array(df['action']).reshape(-1,1))
obj_oh = obj_encoder.fit_transform(np.array(df['object']).reshape(-1,1))
loc_oh = loc_encoder.fit_transform(np.array(df['location']).reshape(-1,1))
df['act_oh'] = list(act_oh)
df['obj_oh'] = list(obj_oh)
df['loc_oh'] = list(loc_oh)

df['class']=df['act_oh']
for i in range(len(df)):
  df['class'].iloc[i]= list(df['act_oh'].iloc[i])+list(df['obj_oh'].iloc[i])+list(df['loc_oh'].iloc[i])

df.head()

,transcription,action,object,location,act_oh,obj_oh,loc_oh,class
0,Turn on the kitchen lights,activate,lights,kitchen,"[1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,Turn up the temperature,increase,heat,none,"[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
2,OK now switch the main language to Chinese,change language,Chinese,none,"[0, 0, 1, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Turn down the bathroom temperature,decrease,heat,washroom,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
4,Change the language,change language,none,none,"[0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [81]:
#pre pos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['transcription'])

data = tokenizer.texts_to_sequences(df['transcription'].values)
data = pad_sequences(data, maxlen=10)

EMBEDDING_DIM= 300
nb_words = len(tokenizer.word_index)+1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
  if word in w2v_set:
    embedding_matrix[i] = word2vec.word_vec(word)
    # print(embedding_matrix[i].shape)
  else:
    embedding_matrix[i] = np.zeros(EMBEDDING_DIM,)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(embedding_matrix.shape)

Null word embeddings: 6
(99, 300)


In [82]:
X = data
Y = pd.DataFrame()

for i in range(23):
  Y[i+1]=df['class'].apply(lambda x: int(x[i]))

VALIDATION_SPLIT = 0.2
perm = np.random.permutation(len(X))
idx_train = perm[:int(len(X)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(X)*(1-VALIDATION_SPLIT)):]
print(len(idx_train), len(idx_val))

X_train = X[idx_train]
X_val = X[idx_val]
Y_train = Y.iloc[idx_train,:]
Y_val = Y.iloc[idx_val,:]

print(X_train.shape, X_val.shape)
print(Y_train.shape, Y_val.shape)

198 50
(198, 10) (50, 10)
(198, 23) (50, 23)


In [83]:
#Model
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=10,
        trainable=False)

lstm_layer_1 = LSTM(20, return_sequences=False, dropout=0.15, recurrent_dropout=0.15)
#lstm_layer_2 = LSTM(200, return_sequences=False, dropout=0.15, recurrent_dropout=0.15)

sequence_input = Input(shape=(10,), dtype='int32')
x1 = embedding_layer(sequence_input)
x1 = lstm_layer_1(x1)
#x1 = lstm_layer_2(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
x1 = Dense(50, activation='relu')(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
preds = Dense(23, activation='sigmoid')(x1)

model = Model(inputs=sequence_input, outputs=preds)
print(model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 10, 300)           29700     
                                                                 
 lstm_1 (LSTM)               (None, 20)                25680     
                                                                 
 dropout_2 (Dropout)         (None, 20)                0         
                                                                 
 batch_normalization_2 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 50)                1050      
                                                           

In [86]:
# Y_train = tf.convert_to_tensor(Y_train)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

loss_fn = tf.keras.losses.MeanAbsoluteError()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
train_acc_metric = tf.keras.losses.MeanAbsoluteError()
val_acc_metric = tf.keras.losses.MeanAbsoluteError()

model.compile(optimizer=optimizer, loss=loss_fn, metrics=train_acc_metric)
model.fit(X_train, Y_train, batch_size=1024, epochs=1000, validation_data=(X_val, Y_val), use_multiprocessing=True)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.2593 - mean_absolute_error: 0.2593 - val_loss: 0.3248 - val_mean_absolute_error: 0.3248
Epoch 2/1000
1/1 [==============================] - 0s 82ms/step - loss: 0.2586 - mean_absolute_error: 0.2586 - val_loss: 0.3218 - val_mean_absolute_error: 0.3218
Epoch 3/1000
1/1 [==============================] - 0s 84ms/step - loss: 0.2565 - mean_absolute_error: 0.2565 - val_loss: 0.3197 - val_mean_absolute_error: 0.3197
Epoch 4/1000
1/1 [==============================] - 0s 94ms/step - loss: 0.2555 - mean_absolute_error: 0.2555 - val_loss: 0.3174 - val_mean_absolute_error: 0.3174
Epoch 5/1000
1/1 [==============================] - 0s 84ms/step - loss: 0.2552 - mean_absolute_error: 0.2552 - val_loss: 0.3153 - val_mean_absolute_error: 0.3153
Epoch 6/1000
1/1 [==============================] - 0s 80ms/step - loss: 0.2527 - mean_absolute_error: 0.2527 - val_loss: 0.3132 - val_mean_absolute_error: 0.3132
Epoch 7/1000
1/1 [======

In [87]:
test_df= pd.read_csv('valid_data.csv')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_df['transcription'])

max_len = 10
data = tokenizer.texts_to_sequences(test_df['transcription'].values)
data = pad_sequences(data, maxlen=max_len)

preds = model.predict(data)
ans=preds
# print(ans)

test_df['act_oh']=test_df['transcription']
test_df['obj_oh']=test_df['transcription']
test_df['loc_oh']=test_df['transcription']

for i in range(len(test_df)):
  temp=[]
  for j in range(6):
    temp.append(round(ans[i][j]))
  test_df['act_oh'].iloc[i]= np.array(temp)

  temp=[]
  for j in range(4,18):
    temp.append(round(ans[i][j]))
  test_df['obj_oh'].iloc[i]= np.array(temp)
  
  temp=[]
  for j in range(19,23):
    temp.append(round(ans[i][j]))
  test_df['loc_oh'].iloc[i]= np.array(temp)

# print(test_df.head())

# test_df.head()

test_df['act_pred']= test_df['act_oh'].apply(lambda x: act_encoder.inverse_transform(x.reshape(1,-1)))
test_df['obj_pred']= test_df['obj_oh'].apply(lambda x: obj_encoder.inverse_transform(x.reshape(1,-1)))
test_df['loc_pred']= test_df['loc_oh'].apply(lambda x: loc_encoder.inverse_transform(x.reshape(1,-1)))

test_df['act_pred']=test_df['act_pred'].apply(lambda x: x[0][0])
test_df['act_pred']=test_df['act_pred'].apply(lambda x: x if x else 'none')

test_df['obj_pred']=test_df['obj_pred'].apply(lambda x: x[0][0])
test_df['obj_pred']=test_df['obj_pred'].apply(lambda x: x if x else 'none')

test_df['loc_pred']=test_df['loc_pred'].apply(lambda x: x[0][0])
test_df['loc_pred']=test_df['loc_pred'].apply(lambda x: x if x else 'none')

# print(test_df['act_pred'].head())
# micro f1 score
action_f1 = f1_score(test_df['act_pred'],test_df['action'], average='micro')
object_f1 = f1_score(test_df['obj_pred'],test_df['object'], average='micro')
location_f1 = f1_score(test_df['loc_pred'],test_df['location'], average='micro')
print('F1 score for action-->',action_f1)
print('F1 score for object-->',object_f1)
print('F1 score for location-->',location_f1)

F1 score for action--> 0.4640795381654907
F1 score for object--> 0.04073123797305965
F1 score for location--> 0.14175753688261705
